In [15]:
import pandas as pd
import shap
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier, SGDClassifier, PassiveAggressiveClassifier
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score
from sklearn.model_selection import train_test_split

In [16]:
def distribution(dataset):
    
    target0 = dataset[dataset == 0]
    target1 = dataset[dataset == 1]
    
    print(f"Target 0: {len(target0)/(len(target0) + len(target1)) * 100} %")
    print(f"Target 1: {len(target1)/(len(target0) + len(target1)) * 100} %")

# Load datasets
train_data = pd.read_csv(f'../dataset_classification/TrainDataset.csv')
test_data = pd.read_csv(f'../dataset_classification/TestDataset.csv')

#Clean TR from features
x_train = train_data.drop(columns = ["label"])
y_train = train_data["label"]

#Split TR in TR and VL with HoldOut technique
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

#Clean TS from features
x_test = test_data.drop(columns = ["label"])
y_test = test_data["label"]

print("Training dataset:")
distribution(y_train)

print("\nValidation dataset:")
distribution(y_val)

Training dataset:
Target 0: 15.418750111215902 %
Target 1: 84.5812498887841 %

Validation dataset:
Target 0: 15.37701855064727 %
Target 1: 84.62298144935274 %


In [ ]:
# Define base classifier
base_estimators = [
    ('logistic', LogisticRegression(max_iter=500, random_state=42)),
    ('ridge', RidgeClassifier(max_iter=500, random_state=42)),
    ('sgd', SGDClassifier(max_iter=500, tol=1e-3, random_state=42)),
    ('passive_aggressive', PassiveAggressiveClassifier(max_iter=500, tol=1e-3, random_state=42))
]

# Define meta classifier
meta_clf = LogisticRegression(max_iter=500, random_state=42)

# Create model
stacking_clf = StackingClassifier(
    estimators=base_estimators,
    final_estimator=meta_clf,
    n_jobs=-1
)

# Train model
print("Starting training for StackingClassifier with linear models...")
stacking_clf.fit(x_train, y_train)
print("Finished training.")

# Evaluate model
print("")
print("Final evaluation")
print("")

train_accuracy = accuracy_score(y_train, stacking_clf.predict(x_train))
val_accuracy = accuracy_score(y_val, stacking_clf.predict(x_val))
test_accuracy = accuracy_score(y_test, stacking_clf.predict(x_test))

print(f"Train accuracy: {train_accuracy}")
print(f"Validation accuracy: {val_accuracy}")
print(f"Test accuracy: {test_accuracy}")
print("")

# Calculate log-loss
train_y_proba = stacking_clf.predict_proba(x_train)
val_y_proba = stacking_clf.predict_proba(x_val)
test_y_proba = stacking_clf.predict_proba(x_test)

train_loss = log_loss(y_train, train_y_proba)
val_loss = log_loss(y_val, val_y_proba)
test_loss = log_loss(y_test, test_y_proba)

print(f"Train Log-Loss: {train_loss}")
print(f"Val Log-Loss: {val_loss}")
print(f"Test Log-Loss: {test_loss}")
print("")

# Calculate AUC
train_auc = roc_auc_score(y_train, train_y_proba[:, 1])
val_auc = roc_auc_score(y_val, val_y_proba[:, 1])
test_auc = roc_auc_score(y_test, test_y_proba[:, 1])

print(f"Train AUC: {train_auc}")
print(f"Val AUC: {val_auc}")
print(f"Test AUC: {test_auc}")
print("")

# SHAP Integration -------------------------------------------------------------------
print("Starting SHAP analysis...")

# Apply SHAP analysis to meta classifier
x_meta_train = stacking_clf.transform(x_train)
explainer = shap.Explainer(stacking_clf.final_estimator_, x_meta_train)
shap_values = explainer(x_meta_train)

# Visualize SHAP analyze to meta classifier
shap.summary_plot(shap_values, x_meta_train)

print("Finished SHAP analysis.")

print("Starting SHAP analysis (foreach model)...")

for model in stacking_clf.estimators_:
    
    # Apply SHAP analysis to base classifier
    explainer = shap.Explainer(model, x_train)
    shap_values = explainer(x_train)
    
    # Visualize SHAP analyze to base classifier
    shap.summary_plot(shap_values, x_train)

Starting training for StackingClassifier with linear models...
